In [109]:
import pandas as pd
import numpy as np
import re
import nltk
#from textblob import TextBlob

import matplotlib.pyplot as plt




In [110]:
#load data from csv/excel/json
data=pd.read_excel('tweets.xlsx') 
#data.head()

In [111]:


#Data preprocessing to removes @usernames,urls,symbols and makes all text lowercase
def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', text)
    text = re.sub('@[^\s]+','', text)
    #text = text.lower().replace("ё", "е")
    text = text.replace("ё", "е")
    dat_str=data.iloc[0]['full_text']
    text = re.sub(dat_str[-10:-7],'\'', text)
    text = re.sub(' &amp','\'', text)
    text = re.sub('\r','', text)
    text = re.sub('\n','', text)
    #print(text)
    #text = re.sub('[^a-zA-Zа-яА-Я1-9]+',' ', text)
    #text = re.sub(' +',' ', text)
    return text.strip()


print(data.iloc[0]['full_text'])
text = [preprocess_text(t) for t in data.iloc[:]['full_text']]
#storing preprocessed texts in a separate column 
data["processed_text"]= text
print(data.iloc[0]["processed_text"])
#print(dat_str[-10:-7])

@ClareTyne @mesainy @HKrassenstein @realDonaldTrump @NYCMayor @JoeBiden Yep.  Torturing that guy because the Dems donâ€™t care.
Yep.  Torturing that guy because the Dems don't care.


In [112]:
#Vader sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()
#calling the Textblob function for each text
def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(snt)))


sentiment_str = [analyser.polarity_scores(t) for t in data.iloc[:]["processed_text"]]

selected = data[["id_str","full_text","processed_text"]]
df=selected.copy()

#getting the polarity and subjectivity of the blob
polarity=[]
for i in range(data.shape[0]):
    polarity.append(sentiment_str[i])
    
df["Polarity/subjectivity"]= polarity

sentiment= []

for j in range(data.shape[0]):
    if polarity[j]['compound']>0.05:
        sentiment.append("Positive")
    elif polarity[j]['compound']<-0.05:
        sentiment.append("Negative")
    else:
        sentiment.append("Neutral")
        
df["Sentiment"]= sentiment
pd.options.display.max_colwidth = 150
df[["processed_text","Sentiment"]]

,processed_text,Sentiment
0,Yep. Torturing that guy because the Dems don't care.,Negative
1,"""Arizona reports record spike in new coronavirus cases'; deaths ahead of Pence's visit"" His megachurch stunt took place 5 days ago in Phoenix.",Neutral
2,Debate President Trump. PROVE you don't have dementia. #DementiaJoeCantDebate #JoeBidenScaredToDebate,Neutral
3,You disrespected our soldiers in Afganistan so your not and better.,Negative
4,Democrats are always saving dinosaurs like grassley after they fvck up,Positive
5,Just found out about you and so far I love your policies and what you have to say. Ofc I still need to read more about your stances and plans but ...,Positive
6,"Dude, get a grip",Neutral
7,This is like the 4th or 5th time this story has come out since Trump took office.,Positive
8,After murders of young people?,Negative
9,The Impeached will always blame others.,Negative
